# WSW Spike Charts

Iterate through folders, create charts for sessions affected by spikes.

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://github.com/Logiqx/gps-wizard) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

## Notes

Sessions have been pre-identified and placed into a seperate folder.

In [52]:
import os
import sys
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

import traceback

corePath = os.path.join('..', 'core')
if corePath not in sys.path:
    sys.path.extend([corePath])

from file_reader import getFileReader

## Main Function

In [53]:
def processFiles():
    '''Iterate through session archive creating charts for each GPS file'''

    rootDir = os.path.join(projdir, '..', 'gps-spikes', 'gpslogs', 'motion', 'wsw')

    errors = {}
    
    for root, subDirs, files in os.walk(rootDir):
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            
            if ext in ['.oao'] and 'STA867SCO_867_20231013_083816' in file:
                filePath = os.path.join(root, file)
                print(filePath)
                reader = getFileReader(filePath)
                try:
                    # Some legacy ESP-GPS files contain bad checksums
                    if ext == '.ubx':
                        reader.load(ignoreChecksums=True)
                    else:
                        reader.load()
                    
                    # Process all tracks within the file
                    for track in reader.tracks:
                        createCharts(track)
                        print('.', end='')
                        
                except Exception:
                    errors[filePath.replace(projdir + '/', '')] = traceback.format_exc()
                    print('E', end='')

    if len(errors) > 0:
        print(os.linesep * 2 + 'Errors:')
        for error in errors:
            print(error)
            print(errors[error])

In [73]:
# Factor to convert m/s to knots
MS_TO_KNOTS = 3600 / 1852

# Filter values
EHPE_THRESHOLD = 2
HACC_THRESHOLD = 25

ubxCharts = [
    {
        'title': 'Speed over Ground (SOG)',
        'label': 'Speed (knots)',
        'field': 'sog',
        'scale': MS_TO_KNOTS
    },
    {
        'title': 'Number of Satellites',
        'label': 'Satellites',
        'field': 'sat',
        'integer': True
    },
    {
        'title': 'Horizontal Dilution of Precision (HDOP)',
        'label': 'HDOP',
        'field': 'hdop'
    },
    {
        'title': 'Positional Dilution of Precision (PDOP)',
        'label': 'PDOP',
        'field': 'pdop'
    },
    {
        'title': 'Horizontal Accuracy (hAcc)',
        'label': 'hAcc (m)',
        'field': 'ehpe'
    },
    {
        'title': 'Speed Accuracy (sAcc)',
        'label': 'sAcc (knots)',
        'field': 'ehve',
        'scale': MS_TO_KNOTS
    }
]

sirfCharts = [
    {
        'title': 'Speed over Ground (SOG)',
        'label': 'Speed (knots)',
        'field': 'sog',
        'scale': MS_TO_KNOTS
    },
    {
        'title': 'Number of Satellites',
        'label': 'Satellites',
        'field': 'sat',
        'integer': True
    },
    {
        'title': 'Horizontal Dilution of Precision (HDOP)',
        'label': 'HDOP',
        'field': 'hdop'
    },
    {
        'title': 'Estimated Horizontal Position Error (EHPE)',
        'label': 'EHPE (m)',
        'field': 'ehpe'
    },
    {
        'title': 'Standard Deviation of Speed (SDOS)',
        'label': 'SDOS (knots)',
        'field': 'ehve',
        'scale': MS_TO_KNOTS
    }
]

def createSubPlot(ax, title, xPoints, yPoints, ylabel, integer, typicalTsDiff):
    '''Create charts for a GPS track'''
    
    # Titles and axis labels
    ax.set_title(title, y=1.01, fontsize=12)
    ax.set_ylabel(ylabel, fontsize=11)   
    ax.xaxis.set_major_formatter(tck.FuncFormatter(lambda x, p: datetime.fromtimestamp(x).strftime('%H:%M')))

    # Trick to ensure missing non-contiguous points are plotted in grey
    ax.plot(xPoints, yPoints, color='silver', linestyle='dotted')

    # Plot contiguous data points in blue
    segmentIdx = -1
    prevTs = 0

    for i in range(len(xPoints)):
        currTs = xPoints[i]
        if i > 0:
            if ((currTs - prevTs) * 1000).astype(int) > typicalTsDiff:
                if segmentIdx >= 0:
                    ax.plot(xPoints[segmentIdx:i], yPoints[segmentIdx:i], color='#1f77b4')
                    segmentIdx = -1
            else:
                if segmentIdx < 0:
                    segmentIdx = i
        prevTs = currTs

    if segmentIdx >= 0:
        ax.plot(xPoints[segmentIdx:i], yPoints[segmentIdx:i], color='#1f77b4')

    # Determine number of tick intervals
    duration = xPoints.max() - xPoints.min()
    tickInterval = 900   
    while duration // tickInterval > 12:
        tickInterval *= 2

    # Determine first and last tick
    firstTick = xPoints.min() // tickInterval * tickInterval
    if firstTick < xPoints.min():
        firstTick += tickInterval
    lastTick = xPoints.max() // tickInterval * tickInterval + 1
    ax.set_xticks(np.arange(firstTick, lastTick, tickInterval))
    
    # Ensure y-axis starts at zero
    ax.set_ylim(ymin=0)
    if yPoints.max() > 1000:
        ax.get_yaxis().set_major_formatter(tck.FuncFormatter(lambda x, p: format(int(x), ',')))
    
    # Ensure y-axis only uses integers for satellites
    if integer:
        ax.yaxis.set_major_locator(tck.MaxNLocator(integer=True))

    # Add grid lines
    ax.grid(which='major', axis='y')                  


def getCharts(filePath):
    '''Get chart definitions'''
    
    if os.path.splitext(filePath)[1].lower() in ['.oao', '.ubx']:
        charts = ubxCharts
    else:
        charts = sirfCharts

    return charts

    
def createCharts(filePath, data, description, pngName):
    '''Create charts for a GPS track'''

    charts = getCharts(filePath)
    numAxs = 0
    for chart in charts:
        if chart['field'] in data:
            numAxs += 1
    
    fig, axs = plt.subplots(numAxs, figsize=(16, 24), dpi=100)
    plt.subplots_adjust(hspace=0.3)
    
    filename = os.path.basename(filePath)
    plt.suptitle('{}'.format(filename), y=0.925, fontsize=18, fontweight='bold')

    if description:
        textStr = 'Filtered by {}'.format(description)
    else:
        textStr = 'Unfiltered'

    fig.text(0.5, 0.905, textStr, fontsize=14, fontweight='bold', horizontalalignment='center')

    # Determine typical timestamp difference in ms - e.g. 200 for Motions
    tsDiffs = ((data['ts'][1:] - data['ts'][:-1]) * 1000).round().astype(int)
    uniqueValues, counts = np.unique(tsDiffs, return_counts=True)
    typicalTsDiff = uniqueValues[np.argmax(counts)]

    # X points is simply the timestamps
    xPoints = data['ts']
    
    # Generate each of the charts
    axsId = 0
    for i in range(len(charts)):
        chart = charts[i]

        if chart['field'] in data:
            yPoints = data[chart['field']]

            if 'integer' in chart and chart['integer']:
                integer = True
            else:
                integer = False

            createSubPlot(axs[axsId], chart['title'], xPoints, yPoints, chart['label'], integer, typicalTsDiff)
            axsId += 1
        
    # Ensure directory for PNG exists
    dirName = os.path.join(os.path.dirname(filePath), 'img', os.path.basename(filePath))
    if not os.path.exists(dirName):
        os.makedirs(dirName)
        
    # Save PNG
    fileName = os.path.join(dirName, pngName + '.png')
    print(f"Saving {fileName}...")
    fig.savefig(fileName, bbox_inches='tight', facecolor='w')

    # Ensure the charts do not appear in Jupyter
    plt.close(fig)


def getFilteredData(charts, unfilteredData, points):
    '''Get filtered data'''
    
    filteredData = {}

    filteredData['ts'] = unfilteredData['ts'][points].flatten()

    for chart in charts:
        if chart['field'] in unfilteredData:
            filteredData[chart['field']] = unfilteredData[chart['field']][points].flatten()
        
    return filteredData

    
def processTrack(filePath, track):
    '''Process a GPS track'''

    filterDesc = ''
    data = {}
    
    # Take a copy of the data and convert m/s to knots, where applicable
    data['ts'] = track.data['ts']

    charts = getCharts(filePath)
    for chart in charts:
        if chart['field'] in track.data:
            if 'scale' in chart:
                data[chart['field']] = track.data[chart['field']] * chart['scale']
            else:
                data[chart['field']] = track.data[chart['field']]

    # Unfiltered
    createCharts(filePath, data, None, 'raw')
    
    # Apply "on fix" filter
    if 'fix' in track.data:
        filterDesc += 'fix > 0'
        points = np.argwhere(track.data['fix'] > 0)       
        data = getFilteredData(charts, data, points)
        createCharts(filePath, data, filterDesc, 'on-fix')
    
    # Apply EHPE filter
    if 'ehpe' in data:
        if os.path.splitext(filePath)[1].lower() in ['.oao', '.ubx']:
            threshold = HACC_THRESHOLD
            field = 'hAcc'
        else:
            threshold = EHPE_THRESHOLD
            field = 'EHPE'

        if filterDesc:
            filterDesc += ' and '
        filterDesc += '{} <= {:.01f} m'.format(field, threshold)

        points = np.argwhere(data['ehpe'] <= threshold)
        data = getFilteredData(charts, data, points)
        createCharts(filePath, data, filterDesc, field.lower())


if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    ignoreChecksums = False
    
    # WSW - GT-31
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/gt-31/wsw/206MALLMAN_113200495_20111018_131156.SBN'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/gt-31/wsw/CONNE2GARRY_133200827_20191009_102438.SBN'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/gt-31/wsw/GARRE67DAVID_103201606_20121009_145611.SBN'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/gt-31/wsw/Hardy41James_932000585_20151006_095149.SBN'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/gt-31/wsw/Jenki36Paul_932000540_20171015_095602.SBN'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/gt-31/wsw/K888_123201112_20121009_085502.SBN'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/gt-31/wsw/Penna65Robin_113200494_20141023_090602.SBN'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/gt-31/wsw/WIGGAWOOKIE_123201104_20121010_100627.SBN'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/gt-31/wsw/WSW 18_932000559_20121009_104151.SBN'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/gt-31/wsw/WSWAFOUR_932000173_20121009_092850.SBN'

    # WSW - Motions
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/motion/wsw/ashore/STA867SCO_867_20231013_083816.oao'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/motion/wsw/ashore/KNI841FIN_841_20231007_103000.oao'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/motion/wsw/ashore/TUR808AND_808_20231013_084118.oao'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/motion/wsw/ashore/CRO632PET_632_20231013_092740.oao'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/motion/wsw/ashore/DUN813ROB_813_20231013_094502.oao'

    # Wing dead reckoning
    #filePath = '/home/jovyan/work/gps-wizard/sessions/brog-wing-spike/2940_2022-10-26-1321.oao'
    
    # Motion spike
    #filePath = '/home/jovyan/work/gps-wizard/sessions/motion-spike/2936_2023-06-14-1155.oao'
    
    # ESP
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211020-esp/BN280E_84CCA86023F8_003.ubx'
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211020-esp/BoomL_83AF2466E48_004.ubx'
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211020-esp/Head_L_7C9EBDFAF5C8_002.ubx'
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211020-esp/Head_R_7C9EBDFAF67C_002.ubx'
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211230-esp/Boom_R_84CCA86023F8_004.ubx'
    filePath = '/home/jovyan/work/gps-wizard/sessions/20211230-esp/BoomL_83AF2466E48_008.ubx' # crazy acc values!
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211230-esp/Head_L_7C9EBDFAF5C8_007.ubx'
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211230-esp/Head_R_7C9EBDFAF67C_006.ubx'
    ignoreChecksums = True
    
    # Manfred 20 Hz
    #filePath = '/home/jovyan/work/gps-wizard/sessions/manfred-20hz/in/093154.UBx'
    #ignoreChecksums = True
    
    # COROS
    #filePath = '/home/jovyan/work/gps-wizard/sessions/wsw-coros-spike/Speedsurfing20221015120552.fit'
    
    reader = getFileReader(filePath)
    print(f"Loading {filePath}...")
    if os.path.splitext(filePath)[1].lower() in ['.sbn', '.oao', '.ubx']:
        reader.load(ignoreChecksums=ignoreChecksums)
    else:
        reader.load()
    track = reader.tracks[0]
    
    processTrack(filePath, track)

Loading /home/jovyan/work/gps-wizard/sessions/20211230-esp/BoomL_83AF2466E48_008.ubx...
Saving /home/jovyan/work/gps-wizard/sessions/20211230-esp/img/BoomL_83AF2466E48_008.ubx/raw.png...
Saving /home/jovyan/work/gps-wizard/sessions/20211230-esp/img/BoomL_83AF2466E48_008.ubx/on-fix.png...
Saving /home/jovyan/work/gps-wizard/sessions/20211230-esp/img/BoomL_83AF2466E48_008.ubx/hacc.png...


In [55]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    #processFiles()
    
    print(os.linesep + 'All done!')


All done!
